In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import os 
import numpy as np
import cv2
from tqdm import tqdm

In [12]:
params = {'input_channels':3, 'dim':768, 'hidden_dim':3072, 'patch_size':16, 'img_size':224, 'num_layers':12, 
          'dropout':0.0, 'attention_dropout':0.0, 'num_heads':12, 'fine_tune':None, 'num_classes':10, 'encoder_norm':True, 'fc_norm':False}

In [13]:
from vit.network import *
from vit.model import *

In [14]:
model = VisionTransformer(**params)

In [15]:
import torchvision
from torchvision import transforms

In [16]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [17]:
data = torchvision.datasets.CIFAR10('D:/Data/random/', transform=transform_train, train=True, download=False)

In [18]:
train, val = torch.utils.data.random_split(data, [40000, 10000])
trainloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=16, num_workers=8)
valloader = torch.utils.data.DataLoader(val, shuffle=True, batch_size=16, num_workers=8)

In [19]:
device = torch.device('cuda:0')

In [20]:
model.load_state_dict(torch.load('10omfg.pth', map_location=device))

<All keys matched successfully>

In [21]:
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.005)
loss = nn.CrossEntropyLoss()

In [22]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2070 SUPER'

In [23]:
model.train()

VisionTransformer(
  (patch_embed): PatchEmbedding(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (blocks): Sequential(
    (0): EncoderBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): MultiheadAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (dropout): Dropout(p=0.0, inplace=False)
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (dropout_1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (dropout_2): Dropout(p=0.0, inplace=False)
      )
    )
    (1): EncoderBlock(
      (norm1): LayerNorm((768,), eps=1e-06, ele

In [24]:
training_losses = []
val_losses = []
avg_training_losses = []
avg_val_losses = []

In [25]:
for e in tqdm(range(10)):
    for x, y in trainloader:
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        y_pred = model(x.float())
        l = loss(y_pred, y)
        l.backward()
        optimizer.step()

        npl = l.detach().cpu().numpy()
        training_losses.append(npl)

    for x, y in valloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x.float())
        l = loss(y_pred, y)

        npl = l.detach().cpu().numpy()
        val_losses.append(npl)

    avg_training_losses.append(np.mean(training_losses))
    avg_val_losses.append(np.mean(val_losses))

    print(f'Epoch: {e}, Training Loss: {np.mean(training_losses)}, Validation Loss: {np.mean(val_losses)}')

  2%|▉                                               | 1/50 [13:36<11:06:55, 816.65s/it]

Epoch: 0, Training Loss: 2.332289218902588, Validation Loss: 2.017587423324585


  4%|█▉                                              | 2/50 [27:18<10:55:45, 819.70s/it]

Epoch: 1, Training Loss: 2.1509275436401367, Validation Loss: 1.9676073789596558


  6%|██▉                                             | 3/50 [41:01<10:43:26, 821.41s/it]

...

Epoch: 47, Training Loss: 1.7629648447036743, Validation Loss: 1.7509987354278564


 98%|██████████████████████████████████████████████ | 49/50 [11:33:53<13:29, 809.73s/it]

Epoch: 48, Training Loss: 1.7614150047302246, Validation Loss: 1.74971604347229


100%|███████████████████████████████████████████████| 50/50 [11:47:23<00:00, 848.86s/it]

Epoch: 49, Training Loss: 1.7593122720718384, Validation Loss: 1.747793197631836


In [26]:
torch.save(model, '50epoch.pth')

In [27]:
model.eval()

VisionTransformer(
  (patch_embed): PatchEmbedding(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (blocks): Sequential(
    (0): EncoderBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): MultiheadAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (dropout): Dropout(p=0.0, inplace=False)
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (dropout_1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (dropout_2): Dropout(p=0.0, inplace=False)
      )
    )
    (1): EncoderBlock(
      (norm1): LayerNorm((768,), eps=1e-06, ele

In [46]:
params = {'input_channels':3, 'dim':768, 'hidden_dim':3072, 'patch_size':16, 'img_size':224, 'num_layers':12, 
          'dropout':0.1, 'attention_dropout':0.1, 'num_heads':12, 'fine_tune':None, 'num_classes':10, 'encoder_norm':True, 'fc_norm':False}

In [47]:
model2 = VisionTransformer(**params)

In [48]:
model2 = model2.to(device)

In [49]:
model2.load_state_dict(torch.load('10omfg.pth', map_location=device))

<All keys matched successfully>

In [57]:
import timm
model2 = timm.create_model('vit_base_patch16_224_in21k', pretrained=True, num_classes=10)
model2 = model2.to(device)

In [53]:
a, b = [], []

for x, y in valloader:
    x, y = x.to(device), y.to(device)
    x = x.float()
    y_pred = model2(x)
    y_pred, y = y_pred.detach().cpu().numpy(), y.detach().cpu().numpy()
    yes = np.argmax(y_pred, axis=1)
    a.append(yes)
    b.append(y)

In [54]:
count = 0
for i, j in zip(a, b):
    for k, l in zip(i, j):
        if k == l:
            count += 1
count/10000

0.0

In [ ]:
# def train(model, optimizer, loss, epochs, trainloader, valloader, device):
#     avg_tloss, avg_vloss = [], []
#     for e in tqdm(range(epochs)):
#         t_loss, v_loss = [], []
#         for x, y in trainloader:
#             optimizer.zero_grad()
#             x, y = x.to(device), y.to(device)
#             y_pred = model(x.float())
#             l = loss(y_pred, y)
#             l.backward()
#             optimizer.step()
            
#             npl = l.detach().cpu().numpy()
#             t_loss.append(npl)
            
#         with torch.no_grad():
#             for x, y in valloader:
#                 x, y = x.to(device), y.to(device)
#                 y_pred = model(x.float())
#                 l = loss(y_pred, y)
                
#                 npl = l.detach().cpu().numpy()
#                 v_loss.append(npl)
    
#         avg_tloss.append(np.mean(t_loss))
#         avg_vloss.append(np.mean(v_loss))

#         print(f'Epoch: {e}, Training Loss: {np.mean(t_loss)}, Validation Loss: {np.mean(v_loss)}')
    
#     return avg_tloss, avg_vloss, model, optimizer

In [ ]:
# tloss, vloss, model, optimizer = train(model, optimizer, loss, 100, trainloader, valloader, device)